In [1]:
import os
# 상위 폴더 이동
import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('ensemble.ipynb'))))

import argparse
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from importlib import import_module

import torch
from datasets.dataset import CustomDataLoader, collate_fn, test_transform
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch.nn.functional as F
import torch.nn as nn

In [2]:
output_dir = './output'
os.makedirs(output_dir, exist_ok=True)

# load sample_submission.csv
submission = pd.read_csv('sample_submission.csv', index_col=None)

In [3]:
print("Start prediction..")
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

test_dataset = CustomDataLoader(data_dir='/opt/ml/segmentation/input/data/test.json', mode='test', transform=test_transform)
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=8,
    num_workers=4,
    shuffle=False,
    pin_memory=use_cuda,
    collate_fn=collate_fn
)

Start prediction..
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


tta augmentation 효과 변경

In [4]:
import ttach as tta
transforms = tta.Compose(
[
    tta.HorizontalFlip(),
    tta.Rotate90(angles=[0, 180]),
    # tta.Scale(scales=[1, 2, 4]),
    tta.Multiply(factors=[0.9, 1, 1.1]),      
])

In [5]:
class interpolate(nn.Module):
    def __init__(self, scale):
        super().__init__()
        self.interp = nn.functional.interpolate
        self.scale = scale

    def forward(self, input):
        return self.interp(input, mode = 'bilinear', scale_factor=self.scale)


In [25]:
import torch.nn.functional as F

models_name = ['TransUnet', 'DeepLabV3_Plus', 'MscaleOCRNet']
ckpt_list = ['../runs/transunet_b16_SGD_1024_fulldata.pt',
            '../runs/deeplab_b7_cp.pt',
            '../runs/augmix_save.pt']

model_module = getattr(import_module("models.model"), models_name[0])
model1 = model_module(
    num_classes=11, pretrained=True
)
checkpoint = torch.load(ckpt_list[0], map_location=device)
model1.load_state_dict(checkpoint)
model1 = model1.to(device)

model_module = getattr(import_module("models.model"), models_name[1])
model2 = model_module(
    num_classes=11, pretrained=True
)
checkpoint = torch.load(ckpt_list[1], map_location=device)
model2.load_state_dict(checkpoint)
model2 = model2.to(device)

model_module = getattr(import_module("models.model"), models_name[2])
model3 = model_module(
    num_classes=11, pretrained=True
)
checkpoint = torch.load(ckpt_list[2], map_location=device)
model3.load_state_dict(checkpoint)
model3 = model3.to(device)

tta_model1 = tta.SegmentationTTAWrapper(model1, transforms)
tta_model2 = tta.SegmentationTTAWrapper(model2, transforms)
# tta_model3 = tta.SegmentationTTAWrapper(model3, transforms)

model1.eval()
tta_model1.eval()
model2.eval()
tta_model2.eval()
model3.eval()
# tta_model3.eval()

load_pretrained: grid-size from 24 to 32
model state dict load skip ocr.cls_head.weight
model state dict load skip ocr.cls_head.bias
model state dict load skip ocr.aux_head.2.weight
model state dict load skip ocr.aux_head.2.bias


MscaleOCRNet(
  (model): MscaleOCR(
    (backbone): HighResolutionNet(
      (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): B

In [ ]:
models

In [26]:
size = 256
transform = A.Compose([A.Resize(size, size)])

file_name_list = []
preds_array = np.empty((0, size*size), dtype=np.compat.long)

In [27]:
test_dataset[50]

(tensor([[[0.6667, 0.6902, 0.7020,  ..., 0.3412, 0.3608, 0.3725],
          [0.6667, 0.6941, 0.7098,  ..., 0.3294, 0.3647, 0.3922],
          [0.5608, 0.6745, 0.7333,  ..., 0.2745, 0.2902, 0.3059],
          ...,
          [0.1843, 0.1686, 0.1529,  ..., 0.5529, 0.6039, 0.6000],
          [0.1137, 0.1137, 0.1294,  ..., 0.6039, 0.6431, 0.6667],
          [0.1725, 0.1569, 0.1608,  ..., 0.4824, 0.4745, 0.5137]],
 
         [[0.4314, 0.4431, 0.4510,  ..., 0.3098, 0.3294, 0.3412],
          [0.4314, 0.4588, 0.4588,  ..., 0.2980, 0.3333, 0.3608],
          [0.3333, 0.4431, 0.4902,  ..., 0.2431, 0.2588, 0.2745],
          ...,
          [0.1686, 0.1529, 0.1373,  ..., 0.5569, 0.5961, 0.6039],
          [0.0980, 0.0980, 0.1137,  ..., 0.5961, 0.6275, 0.6588],
          [0.1569, 0.1412, 0.1451,  ..., 0.4667, 0.4588, 0.4980]],
 
         [[0.4000, 0.4157, 0.4157,  ..., 0.2667, 0.2863, 0.2980],
          [0.3922, 0.4196, 0.4235,  ..., 0.2549, 0.2902, 0.3176],
          [0.2863, 0.3882, 0.4392,  ...,

In [ ]:
with torch.no_grad():
    for step, (imgs, image_infos) in enumerate(tqdm(test_loader)):
        # inference (512 x 512)
        outs = 0
        for model_name in models_name:
            if model_name in ('FCNRes50', 'FCNRes101', 'DeepLabV3_Res50', 'DeepLabV3_Res101'):
                # output size : ([8, 11, 512, 512])
                outs += model1(torch.stack(imgs).to(device))['out']
                outs += tta_model1(torch.stack(imgs).to(device))['out']
                # print('if : ', model_name)
            else:
                # 2. 원하는 모델을 더해주고 더해준 모델의 개수만큼 나눠준다.
                images = torch.stack(imgs).to(device)
                
#                 # transunet input size 변경 
                image1 = []
                for i in imgs:
                    i = i.squeeze()
                    i = torch.transpose(i, 0,2)
                    i1 = cv2.resize(np.float32(i), (1024,1024), interpolation = cv2.INTER_CUBIC)
                    i1 = torch.transpose(torch.tensor(i1), 0,2)
                    image1.append(i1)
                transunet_image1 = torch.stack(image1).to(device)


                outs += model1(transunet_image1)
                outs += tta_model1(transunet_image1)

                outs += model2(images)
                outs += tta_model2(images)

                outs += model3(images)['pred']
                # outs += tta_model3(images)

                # print('else : ', model_name)
                # outs = (1/4)*(model1(images)+tta_model1(images)+model2(images)+tta_model2(images))   
                
            outs = (1/5)*outs ### 모델 개수만큼 바꿔줘야 됨!

        oms = torch.argmax(outs.squeeze(), dim=1).detach().cpu().numpy() # (8, 512, 512)

        # resize (256 x 256)
        temp_mask = []
        for img, mask in zip(np.stack(imgs), oms):
            transformed = transform(image=img, mask=mask)
            mask = transformed['mask']
            temp_mask.append(mask)


        oms = np.array(temp_mask)
        oms = oms.reshape([oms.shape[0], size*size]).astype(int)
        preds_array = np.vstack((preds_array, oms))

        file_name_list.append([i['file_name'] for i in image_infos])

print("End prediction!")
file_names = [y for x in file_name_list for y in x]

In [ ]:
# write PredictionString
for file_name, string in zip(file_names, preds_array):
    submission = submission.append({"image_id": file_name, "PredictionString": ' '.join(str(e) for e in string.tolist())},
                                    ignore_index=True)

# save submission.csv
submission.to_csv(output_dir+'/submission.csv', index=False)